In [1]:
import pandas as pd
from pymongo import MongoClient
import re
import numpy as np
from Functions.functions import *

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [3]:
companies = db.companies.find({
    "offices":{
        "$not":{
            "$size":0
        }
    }
, "founded_year":{
    "$gte": 2007
}
}
,{"name":1, "founded_year":1, "offices":1, "category_code":1, "total_money_raised":1, "deadpooled_year":1, "number_of_employees":1})


In [4]:
data_companies = pd.DataFrame(companies)
data_companies.head()

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,total_money_raised,offices
0,52cdef7c4bab8bd675297d97,Scribd,news,50.0,2007,NaN,$25.8M,"[{'description': 'HQ', 'address1': '539 Bryant..."
1,52cdef7c4bab8bd675297d9c,MeetMoi,social,15.0,2007,NaN,$5.58M,"[{'description': None, 'address1': '', 'addres..."
2,52cdef7c4bab8bd675297da0,Babelgum,games_video,NaN,2007,2013.0,$13.2M,"[{'description': '', 'address1': '', 'address2..."
3,52cdef7c4bab8bd675297daa,Sparter,games_video,NaN,2007,2008.0,$0,"[{'description': None, 'address1': None, 'addr..."
4,52cdef7c4bab8bd675297da9,Mahalo,web,40.0,2007,NaN,$21M,"[{'description': '', 'address1': '3525 Eastham..."


In [5]:
geo_office = data_companies[["offices"]].apply(getGeoLocation, result_type="expand", axis=1) #Aplico la función

In [6]:
df_clean = pd.concat([data_companies,geo_office], axis=1)[["principal_office"]]

In [7]:
data_companies['geoDescription'] = df_clean #Creo la columna

data_companies.head()

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,total_money_raised,offices,geoDescription
0,52cdef7c4bab8bd675297d97,Scribd,news,50.0,2007,NaN,$25.8M,"[{'description': 'HQ', 'address1': '539 Bryant...","{'type': 'Point', 'coordinates': [-122.404052,..."
1,52cdef7c4bab8bd675297d9c,MeetMoi,social,15.0,2007,NaN,$5.58M,"[{'description': None, 'address1': '', 'addres...","{'type': 'Point', 'coordinates': [-73.985506, ..."
2,52cdef7c4bab8bd675297da0,Babelgum,games_video,NaN,2007,2013.0,$13.2M,"[{'description': '', 'address1': '', 'address2...","{'type': 'Point', 'coordinates': [-6.267494, 5..."
3,52cdef7c4bab8bd675297daa,Sparter,games_video,NaN,2007,2008.0,$0,"[{'description': None, 'address1': None, 'addr...","{'type': 'Point', 'coordinates': [-95.712891, ..."
4,52cdef7c4bab8bd675297da9,Mahalo,web,40.0,2007,NaN,$21M,"[{'description': '', 'address1': '3525 Eastham...","{'type': 'Point', 'coordinates': [-118.487267,..."


In [8]:
data_companies['category_code'].value_counts()

web                 1167
software             514
games_video          365
advertising          287
mobile               277
ecommerce            222
other                222
enterprise           213
network_hosting      160
public_relations     154
consulting           141
search               115
cleantech             61
biotech               53
hardware              37
social                29
analytics             29
security              24
education             19
news                  18
music                 18
finance               16
travel                14
photo_video           14
messaging             12
health                 8
sports                 8
medical                6
semiconductor          6
legal                  6
fashion                5
real_estate            4
hospitality            4
design                 3
manufacturing          3
nanotech               1
nonprofit              1
automotive             1
transportation         1
Name: category_code, dtyp

In [9]:
list_category = {'web': 'tech', 'software': 'tech', 
                 'games_video':'tech', 'advertising':'tech', 'mobile':'tech', 'ecommerce':'tech', 'search':'tech', 'network_hosting':'tech'
                ,'consulting':'tech','hardware':'tech', 'biotech':'tech', 'cleantech':'tech', 'analytics':'tech','hardware':'tech', 'photo_video':'tech',
                'messaging':'tech', 'design':'tech'}

In [10]:
data_companies = data_companies.replace(to_replace=list_category, inplace=False)

In [11]:
#Reemplazo las categorías para quedarme con la categoría de tech en los casos de arriba y el resto mantenerlas como son

data_companies['founded_year'] = data_companies['founded_year'].dropna(axis=0).astype('int64', inplace=True)

In [12]:
latitude = [d[0].get('latitude') for d in data_companies.offices]
longitude = [d[0].get('longitude') for d in data_companies.offices]

In [13]:
data_companies['latitude'] = latitude
data_companies['longitude'] = longitude

In [14]:
city = [d[0].get('city') for d in data_companies.offices]
country = [d[0].get('country_code') for d in data_companies.offices]

In [15]:
#seleciono la ciuidad y el código de país para crear dos columnas

data_companies['city'] = city
data_companies['country'] = country
data_companies.head()

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,total_money_raised,offices,geoDescription,latitude,longitude,city,country
0,52cdef7c4bab8bd675297d97,Scribd,news,50.0,2007,NaN,$25.8M,"[{'description': 'HQ', 'address1': '539 Bryant...","{'type': 'Point', 'coordinates': [-122.404052,...",37.789634,-122.404052,San Francisco,USA
1,52cdef7c4bab8bd675297d9c,MeetMoi,social,15.0,2007,NaN,$5.58M,"[{'description': None, 'address1': '', 'addres...","{'type': 'Point', 'coordinates': [-73.985506, ...",40.757929,-73.985506,New York City,USA
2,52cdef7c4bab8bd675297da0,Babelgum,tech,NaN,2007,2013.0,$13.2M,"[{'description': '', 'address1': '', 'address2...","{'type': 'Point', 'coordinates': [-6.267494, 5...",53.344104,-6.267494,London,GBR
3,52cdef7c4bab8bd675297daa,Sparter,tech,NaN,2007,2008.0,$0,"[{'description': None, 'address1': None, 'addr...","{'type': 'Point', 'coordinates': [-95.712891, ...",37.090240,-95.712891,None,USA
4,52cdef7c4bab8bd675297da9,Mahalo,tech,40.0,2007,NaN,$21M,"[{'description': '', 'address1': '3525 Eastham...","{'type': 'Point', 'coordinates': [-118.487267,...",34.017606,-118.487267,Culver City,USA


In [16]:
def dropcolumns(data, columns):
        data = data.drop(columns, axis=1)
        return data

In [17]:
data_companies = dropcolumns(data_companies, ['offices', '_id']) #Borro las columnas de office y _id porque no las necesito

In [18]:
data_companies = data_companies[data_companies['deadpooled_year'].isna()] 


In [19]:
data_companies['city'].value_counts()

San Francisco                      277
New York                           221
                                   203
London                             147
Seattle                             69
                                  ... 
Heredia                              1
Whistler                             1
Warminster                           1
Great Falls                          1
Ciudad Autonoma de Buenos Aires      1
Name: city, Length: 1136, dtype: int64

In [20]:
data_companies['city'] = data_companies['city'].apply(cleanLocation)
#Limpio las localizaciones para terminar de unificarlas

In [21]:
#Elimino nulos de estas columnas porque no me sirven empresas que no sepamos el año de fundación
#ni la latitud o longitud

data_companies.dropna(subset=['founded_year', 'latitude', 'longitude'], inplace=True)

In [22]:
data_companies = dropcolumns(data_companies, 'deadpooled_year')

#Ya me quedé con las empresas que tienen actividad, así que no necesito más la columna


In [23]:
#Función para unificar las monedas y su valor en miles, millones o billions



data_companies['total_money_raised'] = data_companies['total_money_raised'].apply(moneyRaise)


In [24]:
#Rellenar los nulos en la columna de empleados

def fillNaN(data, col):
    [data[col].fillna(0, inplace=True) for col in data.columns]
    return data

In [25]:
data_companies = fillNaN(data_companies, 'number_of_employees')

In [26]:
data_companies = data_companies[data_companies['number_of_employees'] != 0.0]
data_companies = data_companies[data_companies['total_money_raised']!=0]

#Quiero empresas que tengan al menos un empleado o algún tipo de ingresos


In [27]:
ata_companies = data_companies[data_companies['country'] != 'USA']

In [28]:
data_companies = data_companies.reset_index(drop=True)

data_companies.head()

,name,category_code,number_of_employees,founded_year,total_money_raised,geoDescription,latitude,longitude,city,country
0,Scribd,news,50.0,2007,25800000,"{'type': 'Point', 'coordinates': [-122.404052,...",37.789634,-122.404052,San Francisco,USA
1,MeetMoi,social,15.0,2007,5580000,"{'type': 'Point', 'coordinates': [-73.985506, ...",40.757929,-73.985506,New York City,USA
2,Mahalo,tech,40.0,2007,21000000,"{'type': 'Point', 'coordinates': [-118.487267,...",34.017606,-118.487267,Culver City,USA
3,OpenX,tech,305.0,2008,75500000,"{'type': 'Point', 'coordinates': [-118.1327468...",34.149471,-118.132747,Pasadena,USA
4,AdaptiveBlue,tech,15.0,2007,24000000,"{'type': 'Point', 'coordinates': [-74.3372, 40...",40.801358,-74.337200,NYC,USA


In [29]:
data_companies.to_json('./data_companies_clean.json', orient='records')